In [1]:
#mkdirs()
#capture()
#monitor(maxloops=2,delay=10)
#temp_plot()
#power_plot()
#spectra_plot()
#thermog()
#spectros()
#thermom()
#progressupdates(maxcycles=20,wait=60)
#progressbar((3,1,2))
#stop()
#sentinel(loops=5)
#reset_thermo()

In [2]:
from matplotlib import pyplot as plt
from IPython.display import clear_output
import os
import datetime
import numpy
import time
import warnings
import RPi.GPIO as GPIO
import subprocess
import sh
warnings.filterwarnings("ignore", module="numpy")
warnings.filterwarnings("ignore", module="GPIO")

started = datetime.datetime.now()
tstart = started.strftime("%Y-%m-%d_%H:%M:%S") # to start on the current time
#tstart = '2016-05-12_14:57:44' # start on a specific time
droot = '/home/brandon/repos/bcduino/' # absolute paths
dbash = droot+'bash/'
ddata = droot+'data/'
dpython = droot+'python/'
dexp = ddata+tstart + '/'
dspectra = dexp+'spectra/'
dtemp = dexp+'temperature/'
dthermo = dexp+'thermograph/'
dosc = dexp+'oscilloscope/'
dplot = dexp+'plots/'

#############
### SETUP ###
#############

# synchronize time on all machines with:
# sudo service ntp stop && sudo ntpdate time.nrc.ca && sudo service ntp start

def setup_gpio():
    ### SET UP RPI.GPIO CONTROL
    GPIO.cleanup()
    GPIO.setmode(GPIO.BOARD)
    GPIO.setup(18, GPIO.OUT)
    GPIO.output(18, GPIO.LOW) ## turn the camera on

def mkdirs():
    #### create the folders that will contain the data
    subprocess.Popen(['mkdir',dexp])
    subprocess.Popen(['mkdir',dspectra])
    subprocess.Popen(['/bin/cp',dbash+'spectra.sh',dspectra])
    
    subprocess.Popen(['mkdir',dtemp])
    subprocess.Popen(['/bin/cp',dbash+'temperature.sh',dtemp])
    
    subprocess.Popen(['mkdir',dthermo])
    subprocess.Popen(['/bin/cp',dbash+'thermograph',dthermo])
    
    subprocess.Popen(['mkdir',dosc])
    subprocess.Popen(['/bin/cp',dbash+'oscilloscope.sh',dosc])
    subprocess.Popen(['/bin/cp',dpython+'scope2.py',dosc])
    
    subprocess.Popen(['mkdir',dplot])
    
    print(tstart)
    print(os.listdir(dexp))

In [3]:
#######################
### STOP EVERYTHING ###
#######################

def stop():
    stop = sh.Command(dbash+'stoprun.sh')
    stop()

####################
### SYSTEM RESET ###
####################

def reset_thermo():
    """ The camera is powered through a relay.  The relay coil is normally-closed,
    as controlled by a PNP BJT with the base tied to GPIO18.  """
    kill = sh.Command('killall')
    try:
        kill('thermograph')
    except:
        print("thermography not running; can't kill...")
    try:
        GPIO.output(18, GPIO.HIGH) ## turn the camera off
    except:
        setup_gpio()
    GPIO.output(18, GPIO.HIGH)
    time.sleep(2) ## wait for two seconds
    GPIO.output(18, GPIO.LOW) ## turn the camera on
    thermog()
    return True

def reset_thermometry():
    kill = sh.Command('killall')
    try:
        kill('temperature.sh','cat')
    except:
        pass
    thermom()
    return True

def reset_spectra():
    return True

def reset_oscilloscope():
    kill = sh.Command('killall')
    try:
        kill('oscilloscope.sh')
    except:
        pass
    oscillos()
    return True

########################
### CAPTURE THE DATA ###
########################

def capture_data():
    run_thermometry()
    #thermog()
    run_spectroscopy()
    run_oscilloscope()

def run_thermometry():
    ### run thermometry - WORKS
    # stty -F /dev/ttyACM0 57600 raw
    # cat /dev/ttyACM0 > temperature/temperaturehistory
    capturetemperature = sh.Command(dtemp+'temperature.sh')
    p = capturetemperature(_bg=True,_cwd=dtemp)
    return True

def run_thermography():
    #### run thermography - WORKS
    #kill = sh.Command('killall')
    #kill('thermograph')
    #capturethermograph = sh.Command(dthermo + 'thermograph')
    #p = capturethermograph(_bg=True,_cwd=dthermo)
    return True
    
def run_spectroscopy():
    #### run spectroscopy - WORKS, but sometimes crashes the USB interface
    ## works great, but I don't know how to handle bash looping at the moment!
    ##doceanoptics = droot+"c/oceanoptics/seabreeze-3.0.11/SeaBreeze/"
    ##capturespectrum = sh.Command(doceanoptics+'sample-code/c/seabreeze-util')
    ##p = capturespectrum("--get-spectrum","--integration-time-us","1000000"
    ##                    ,_env={"LD_LIBRARY_PATH": doceanoptics+"lib"}
    ##                    ,_bg=True
    ##                    ,_out=dspectra+"spectrum_"+datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))
    capturespectra = sh.Command(dspectra + 'spectra.sh')
    p = capturespectra(dspectra,_bg=True)
    return True
    
def run_oscilloscope():
    #capturetrace = sh.Command(dosc + 'oscilloscope.sh')
    #p = capturetrace()
    #subprocess.Popen(dosc+'oscilloscope.sh')
    return True

    
################################
### MONITOR REALTIME LOGGING ###
################################

def sentinel(loops=1):
    loop = 0
    while (loop < loops):
        ntemp = sum([os.path.getsize(dtemp+f) for f in os.listdir(dtemp)])
        nthermo = len(os.listdir(dthermo))
        nspectra = len(os.listdir(dspectra))
        nosc = len(os.listdir(dosc))
        time.sleep(10)
        clear_output()
        print(datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))
        ### monitor temperature by filesize - TESTED
        if ( ntemp == sum([os.path.getsize(dtemp+f) for f in os.listdir(dtemp)]) ):
            print("Thermometry has failed...")
            if (reset_temp()): print("Thermometry restarted.")
            else: print("Thermometry restart failed.")
        else: print("Thermometry is running.")
        ### monitor thermography by number of files - TESTED
        if ( nthermo == len(os.listdir(dthermo)) ):
            print("Thermography has failed...")
            if (reset_thermo()): print("Thermography restarted.")
            else: print("Thermography restart failed.")
        else: print("Thermography is running.")
        ### monitor spectroscopy by number of files - TESTED
        if ( nspectra == len(os.listdir(dspectra)) ):
            print("Spectroscopy has failed...")
            if (reset_spectra()): print("Spectroscopy restarted.")
            else: print("Spectroscopy restart failed.")
        else: print("Spectroscopy is running.")
        if ( nosc == len(os.listdir(dosc)) ):
            print("Oscilloscope has failed...")
            if (reset_oscilloscope()): print("Oscilloscope restarted.")
            else: print("Oscilloscope restart failed.")
        loop = loop + 1
    
def progressupdates(maxcycles,wait):
    cycles = 0
    while (cycles < maxcycles):
        progressbar((3,1,2))
        time.sleep(wait)

def progressbar(num):
    # see how things are REALLY going with the run
    f = 'temperaturehistory' #sorted(os.listdir(dtemp))
    data = numpy.genfromtxt(dtemp+f,delimiter=",",invalid_raise=False, skip_header=500000)
    print(dtemp+f)
    fig = plt.figure()
    ax = fig.add_subplot('111')
    colors = ['blue','red','green','purple','darkgrey']
    for position in range(0,len(num)):
        #print(num)
        #print(position)
        #print(num[position])
        if position == 1:
            ax2 = plt.twinx()
        #print([line[num[position]] for line in data])
        plt.plot([line[num[position]] for line in data],linestyle='None',color=colors[position],
                 marker='o',markersize=1,markeredgecolor=colors[position],zorder=position)
    ax.set_title('experimental run progress')
    ax.set_xlabel('counts, @100Hz',fontsize=12)
    ax.tick_params(axis='both',which='major',labelsize=10)
    ax.set_zorder(ax2.get_zorder()+1) # put ax in front of ax2 
    ax.patch.set_visible(False) # hide the 'canvas'
    ax2.set_ylim([15,40])
    fig.savefig(dexp+'progress.png',dpi=150)
    #fig.show()
    plt.close(fig)

def monitor(maxloops, delay):
    """here is a function description"""
    loops = 0
    while (loops < maxloops):

        runtime = (datetime.datetime.now()-started) #.strftime("%Y-%m-%d_%H:%M:%S")
        print("after running for " + str(runtime).split('.')[0] + ":")

        ### check to make sure logging is occurring
        if (len(os.listdir(dspectra)) < 3):
            print("WARNING: spectroscopy down")
        else:
            print(str(len(os.listdir(dspectra))) + " spectra")

        if (len(os.listdir(dthermo)) < 3):
            print("WARNING: thermography down")
        else:
            print(str(len(os.listdir(dthermo))) + " thermographs")

        # not happening on the Pi3 yet
        #if (len(os.listdir(dosc)) < 3):             
        #    print("WARNING: oscilloscope down") 
        #else:
        #    print(len(os.listdir(dosc)) + " oscilloscope traces so far!")

        if (len(numpy.genfromtxt(dtemp+os.listdir(dtemp)[0])) < 10):
            print("WARNING: thermometry down")
        else:
            print(str(len(numpy.genfromtxt(dtemp+os.listdir(dtemp)[0]))) + " temperature datapoints")

        print('')
        time.sleep(delay) # give it a few seconds to log some more stuff
        loops = loops + 1

def standardrun():
    mkdirs()
    setup_gpio()
    capture_data()
    #sentinel(loops=5000)

In [4]:
#standardrun()
mkdirs()

2016-06-03_10:44:52
['thermograph', 'oscilloscope', 'temperature', 'spectra']


In [5]:
capture_data()

In [6]:
stop()

ErrorReturnCode_1: 

  RAN: '/home/brandon/repos/bcduino/bash/stoprun.sh'

  STDOUT:


  STDERR:
thermograph: no process found
